In [ ]:
import numpy as np, keras, os, tensorflow as tf, scipy.misc
from PIL import Image
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, BatchNormalization, ZeroPadding2D
from keras.layers import TimeDistributed, Conv2D, MaxPooling2D, Dense, Dropout, Flatten, Activation, LeakyReLU, Input
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.utils import to_categorical

In [ ]:
x_train, y_train = np.load('data/x_train_op.npy',allow_pickle=True), np.load('data/y_train_op.npy')
x_val,   y_val   = np.load('data/x_dev_op.npy',allow_pickle=True),   np.load('data/y_dev_op.npy')
x_test,  y_test  = np.load('data/x_test_op.npy',allow_pickle=True),  np.load('data/y_test_op.npy')

CLASSES = max(y_train) + 1
name    = 'models/desired_name.hdf5'

In [ ]:
class DataGenerator(keras.utils.Sequence) :
  
    def __init__(self, images, labels, batch_size) :
        self.images = images
        self.labels = labels
        self.batch_size = batch_size
    
    
    def __len__(self) :
        return (np.ceil(len(self.images) / float(self.batch_size))).astype(np.int)
  
  
    def __getitem__(self, idx) :
        
        batch_x = self.images[idx * self.batch_size : (idx+1) * self.batch_size]
        batch_y = self.labels[idx * self.batch_size : (idx+1) * self.batch_size]
        
        return np.expand_dims(np.array(batch_x[0]),0),  np.expand_dims(np.squeeze([to_categorical(batch_y,CLASSES)]*len(batch_x[0])),0)

In [ ]:
def conv_model(INPUT_SIZE):
    
    model = Sequential()

    model.add(Conv2D(64,kernel_size=(3,3),padding='same',strides=1,input_shape = INPUT_SIZE))
    model.add(Conv2D(64,kernel_size=(3,3),padding='same',strides=1))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.3))
    model.add(MaxPooling2D(pool_size=(3,3)))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(128,kernel_size=(3,3),padding='same',strides=1))
    model.add(Conv2D(128,kernel_size=(3,3),padding='same',strides=1))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.3))
    model.add(MaxPooling2D(pool_size=(3,3)))
    model.add(Dropout(0.2))

    model.add(Conv2D(256,kernel_size=(3,3),padding='same',strides=1))
    model.add(Conv2D(256,kernel_size=(3,3),padding='same',strides=1))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.3))
    model.add(MaxPooling2D(pool_size=(3,3)))
    model.add(Dropout(0.2))

    model.add(Conv2D(640,kernel_size=(3,3),padding='same',strides=1))
    model.add(Conv2D(640,kernel_size=(3,3),padding='same',strides=1))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.3))
    model.add(MaxPooling2D(pool_size=(3,3)))
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.3))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))

    model.summary()
    return model

def lstm_model(INPUT_SIZE,classes):
    
    model = Sequential()
    #conv = conv_model(INPUT_SIZE)
    shape = (None,) + (INPUT_SIZE)

    model.add(Bidirectional(LSTM(256,return_sequences=True), input_shape=shape))
    model.add(Dropout(0.2))

    model.add(TimeDistributed(Dense(classes,activation='softmax')))
    
    model.summary()
    return model

In [ ]:
# SOS: Works only for batch_size = 1 !
train_generator    = DataGenerator(images=x_train, labels=y_train, batch_size=1)
val_generator      = DataGenerator(images=x_val,   labels=y_val,   batch_size=1)
test_generator     = DataGenerator(images=x_test,  labels=y_test,  batch_size=1)

In [ ]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

config = tf.ConfigProto( device_count = {'GPU': 0})
config.gpu_options.allow_growth = True 
session = tf.Session(config=config)

In [ ]:
model = lstm_model((201,),CLASSES)

In [ ]:
from keras.callbacks import Callback
class TestCallback(Callback):
    def __init__(self, test_data):
        self.test_data = test_data

    def on_epoch_end(self, epoch, logs={}):
        loss, acc = self.model.evaluate_generator(test_generator)
        print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

In [ ]:
earlystop = EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='min',restore_best_weights=True)
mcp_save = ModelCheckpoint(name, save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, epsilon=1e-4, mode='min')
    
model.compile(loss='categorical_crossentropy',optimizer=keras.optimizers.SGD(),metrics=['accuracy'])
model.fit_generator(generator=train_generator,epochs=200,validation_data = val_generator,
                    verbose = 1,callbacks=[earlystop,mcp_save,reduce_lr_loss,TestCallback(test_generator)])

In [ ]:
model = keras.models.load_model(name)
model.evaluate_generator(test_generator)